In [221]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [222]:
url= 'https://en.wikipedia.org/wiki/London'
url1 = 'https://en.wikipedia.org/wiki/Berlin'
url2= 'https://en.wikipedia.org/wiki/Hamburg'
url3= 'https://en.wikipedia.org/wiki/Munich'


### Parsing

In [223]:
response_l = requests.get(url)
response_l

<Response [200]>

In [224]:
response_b = requests.get(url1)
berlin = BeautifulSoup(response_b.content, 'html.parser')

In [225]:
response_h = requests.get(url2)
hamburg = BeautifulSoup(response_h.content, 'html.parser')

In [226]:
response_m = requests.get(url3)
munich = BeautifulSoup(response_m.content, 'html.parser')

In [227]:
print(berlin.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-appearance-disabled vector-feature-appearance-pinned-clientpref-0 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Berlin - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vect

### Scraping for population

In [228]:
pop_london= london.find_all(class_='infobox-header' )[2]
lon_pop= pop_london.find_next(class_= 'infobox-data').get_text()
lon_pop

'8,799,800[1]'

In [229]:
pop_berlin= berlin.find_all(class_= 'infobox-header')[2]
stat_berlin= pop_berlin.find_next(class_='infobox-data')
ber_pop= stat_berlin.get_text()
ber_pop

'3,755,251'

In [230]:
pop_hamburg= hamburg.find_all(class_= 'infobox-header')[2]
stat_hamburg= pop_hamburg.find_next(class_='infobox-data')
ham_pop= stat_hamburg.get_text()
ham_pop

'1,945,532'

In [231]:
pop_munich= munich.find_all(class_= 'infobox-header')[2]
stat_munich= pop_munich.find_next(class_='infobox-data')
mun_pop=stat_munich.get_text()
mun_pop

'1,512,491'

##### Alternative methods of scraping for population

In [232]:
berlin_population= berlin.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(21) > td')[0].get_text()
berlin_population

'3,755,251'

In [233]:
hamburg_population = hamburg.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(19) > td')[0].get_text()
hamburg_population

'1,945,532'

In [234]:
munich_population= munich.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox.ib-settlement.vcard > tbody > tr:nth-child(21) > td')[0].get_text()
munich_population

'1,512,491'

### Scraping for Longitude and Latitude

##### London

In [235]:
for coords in london.find_all(class_= 'infobox-full-data'):
    if coords.find(string =re.compile ('Coordinate', re.IGNORECASE)):
        lat= coords.find(class_= 'latitude').get_text()
        long = coords.find(class_= 'longitude').get_text()
        print(lat)
        print(long)

51°30′26″N
0°7′39″W


##### Berlin

In [236]:
for coords in berlin.find_all(class_= 'infobox-full-data'):
    if coords.find(string=re.compile('Coordinate', re.IGNORECASE)):
        lat_b = coords.find(class_= 'latitude').get_text()
        long_b = coords.find(class_= 'longitude').get_text()
        print(lat_b)
        print(long_b)

52°31′12″N
13°24′18″E


##### Hamburg

In [237]:
for coords in hamburg.find_all(class_= 'infobox-full-data'):
    if coords.find(string=re.compile('Coordinates', re.IGNORECASE)):
        lat_h= coords.find(class_='latitude').get_text()
        long_h = coords.find(class_='longitude').get_text()
        print(lat_h)
        print(long_h)

53°33′N
10°00′E


##### Munich

In [238]:
for coords in munich.find_all(class_= 'infobox-full-data'):
    if coords.find(string=re.compile('Coordinates', re.IGNORECASE)):
        lat_m= coords.find_next(class_='latitude').get_text()
        long_m = coords.find_next(class_='longitude').get_text()
        print(lat_m)
        print(long_m)

48°08′15″N
11°34′30″E


### Scraping for City

In [239]:
city= london.find('h1').get_text()
city

'London'

In [240]:
city1= berlin.find('h1').get_text()
city1

'Berlin'

In [241]:
city2= hamburg.find('h1').get_text()
city2

'Hamburg'

In [242]:
city3= munich.find('h1').get_text()
city3

'Munich'

### Scraping for Country

In [243]:
country_b= berlin.find_all(class_='infobox-label')[0]
country_= country_b.find_next(class_= 'infobox-data')
Country= country_.get_text()
Country

'Germany'

In [244]:
country_l= london.find_all(class_='infobox-label')[0]
country_= country_l.find_next(class_= 'infobox-data')
Country= country_.get_text()
Country

'United Kingdom'

### Creating Dataframe for Results

##### Manually

In [270]:
df ={
    'city':['Berlin','Hamburg','Munich','London'],
    "population": ['3,755,251', '1,945,532','1,512,491','8,799,800'],
    "latitude":['52°31′12″N','53°33′N', '48°08′15″N','51°30′26″N'], 
    "longitude": ['13°24′18″E','10°00′E','11°34′30″E','0°7′39″W'], 
    "country":['Germany','Germany','Germany', 'United Kingdom']}
pd.DataFrame(df)

,city,population,latitude,longitude,country
0,Berlin,"3,755,251",52°31′12″N,13°24′18″E,Germany
1,Hamburg,"1,945,532",53°33′N,10°00′E,Germany
2,Munich,"1,512,491",48°08′15″N,11°34′30″E,Germany
3,London,"8,799,800",51°30′26″N,0°7′39″W,United Kingdom


##### Looping through

In [268]:
cities = ['berlin','london', 'hamburg', 'munich']
city_df = {'city':[],
          'population':[],
         'latitude':[],
         'longitude':[],
         'country':[]}

for city in cities:
    
    url= 'https://en.wikipedia.org/wiki/'+city
    response = requests.get(url)
    pop_soup = BeautifulSoup(response.content, 'html.parser')
    city = pop_soup.find('h1').get_text()


    for header in pop_soup.find_all(class_='infobox-header'):
         if header.find(string=re.compile("population", re.IGNORECASE)):
            population= header.find_next(class_= 'infobox-data').get_text()
        
    for coords in pop_soup.find_all(class_='infobox-full-data'):
        if coords.find(string=re.compile("Coordinates", re.IGNORECASE)):
            longi = coords.find(class_='longitude').get_text()
                    
    for coords in pop_soup.find_all(class_= 'infobox-full-data'):
        if coords.find(string=re.compile("Coordinates", re.IGNORECASE)):
            lat= coords.find(class_= 'latitude').get_text()
                    
    for label in pop_soup.find_all(class_='infobox-label'):
         if label.find(string=re.compile("population", re.IGNORECASE)):
            country = label.find_next(class_= 'infobox-data').get_text()
                   
    city_df['city'].append(city)
    city_df['population'].append(population)
    city_df['latitude'].append(lat)
    city_df['longitude'].append(longi)
    city_df['country'].append(country)
            
pop_df = pd.DataFrame(city_df)
    

In [269]:
pop_df

,city,population,latitude,longitude,country
0,Berlin,"3,755,251",52°31′12″N,13°24′18″E,England
1,London,"8,799,800[1]",51°30′26″N,0°7′39″W,England
2,Hamburg,"1,945,532",53°33′N,10°00′E,England
3,Munich,"1,512,491",48°08′15″N,11°34′30″E,England


### Putting data in a function

In [259]:
def city_pop(city):
    pop_df = {'city':[],
          'population':[],
         'latitude':[],
         'longitude':[],
         'country':[]}
    
    url = "https://en.wikipedia.org/wiki/"+city
    response = requests.get(url)
    soup_city = BeautifulSoup(response.content, 'html.parser')
    city = soup_city.find('h1').get_text()
    
    for header in soup_city.find_all(class_="infobox-header"):
        if header.find(string=re.compile("population", re.IGNORECASE)):
            popu = header.find_next(class_="infobox-data").text

    for label in soup_city.find_all(class_="infobox-label"):
        if label.find(string=re.compile("country", re.IGNORECASE)):
            country = label.find_next(class_="infobox-data").text
    
    for coords in soup_city.find_all(class_="infobox-full-data"):
        if coords.find(string=re.compile("Coordinates", re.IGNORECASE)):
            lati = coords.find_next(class_="latitude").text
            longi = coords.find_next(class_="longitude").text
            
    pop_df['city'].append(city)
    pop_df['population'].append(popu)
    pop_df['latitude'].append(lat)
    pop_df['longitude'].append(longi)
    pop_df['country'].append(country)
    
    pop_df = pd.DataFrame(pop_df)
    return pop_df

In [265]:
city_pop("Berlin")
city_df = city_pop("London")


In [267]:
city_df['population'] = city_df['population'].str.replace('[1]','')
city_df['population'] = city_df['population'].str.replace(',','')
city_df["population"] = pd.to_numeric(city_df["population"])
city_df

,city,population,latitude,longitude,country
0,London,8799800,48°08′15″N,0°7′39″W,England
